# cadCAD Tutorials: The Robot and the Marbles Part 5 - Networks

To expand upon our previous examples, we will introduce the concept of using a graph network object that is updated during each state update. The ability to essential embed a graph 'database' into a state is a game changer for scalability, allowing increased complexity with multiple agents or components is represented, easily updated. Below, building upon our previous examples, we will represent the Robots and Marbles example with n boxes, and a variable number of marbles. 

## Behavior and Mechanisms:
* A network of robotic arms is capable of taking a marble from their one of their boxes and dropping it into the other one. 
* Each robotic arm in the network only controls two boxes and they act by moving a marble from one box to the other.
* Each robotic arm is programmed to take one marble at a time from the box containing the most significant number of marbles and drop it in the other box. It repeats that process until the boxes contain an equal number of marbles.
* For our analysis of this system, suppose we are only interested in monitoring the number of marbles in only their two boxes.

In [1]:
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
from cadCAD.configuration import Configuration
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
seed = 12302
from copy import deepcopy
np.random.seed(seed)

%matplotlib inline

# define global variables
T = 25 #iterations in our simulation
boxes=5 #number of boxes in our network
m= 2 #for barabasi graph type number of edges is (n-2)*m

We create a [Barabási–Albert](https://en.wikipedia.org/wiki/Barab%C3%A1si%E2%80%93Albert_model) graph and then fill the 5 boxes with between 1 and 10 balls. You can create as many different nodes or types of nodes as needed

In [2]:
# create graph object with the number of boxes as nodes
network = nx.barabasi_albert_graph(boxes, m, seed=seed)

# add balls to box nodes
for node in network.nodes:
    network.nodes[node]['balls'] = np.random.randint(1,10)

In [3]:
# we initialize the cadCAD state as a network object
# we add state variables for metrics of the graph we're interested in
initial_conditions = {'size':network.number_of_nodes(), 'center':nx.center(network)}

In [4]:
def grow_network(params, step, sL, s):
    network = params['network'] #deepcopy(params['network'])
    new_node = network.number_of_nodes()
    network.add_edge(new_node, np.random.randint(0,new_node))
    network.nodes[new_node]['balls'] = np.random.randint(1,10)
    return {'key':0}

def calculate_size(params, step, sL, s, _input):
    return('size', params['network'].number_of_nodes())

def calculate_center(params, step, sL, s, _input):
    return('center', nx.center(params['network']))

In [5]:
# wire up the mechanisms and states
partial_state_update_blocks = [
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
            'network': grow_network
        },
        'variables': { # The following state variables will be updated simultaneously
        }
    },
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
        },
        'variables': { # The following state variables will be updated simultaneously
            'size': calculate_size,
            'center': calculate_center
        }
    }
]

In [6]:
params= {
    'network': [network]
    }
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Settings of general simulation parameters, unrelated to the system itself
# `T` is a range with the number of discrete units of time the simulation will run for;
# `N` is the number of times the simulation will be run (Monte Carlo runs)
time_periods_per_run = 5
monte_carlo_runs = 3

from cadCAD.configuration.utils import config_sim
simulation_parameters = config_sim({
    'T': range(time_periods_per_run),
    'N': monte_carlo_runs,
    'M': params
})

In [7]:
from cadCAD.configuration import append_configs
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# The configurations above are then packaged into a `Configuration` object
append_configs(
    initial_state=initial_conditions, #dict containing variable names and initial values
    partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
    sim_configs=simulation_parameters #dict containing simulation parameters
)
from tabulate import tabulate
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
from cadCAD import configs
import pandas as pd

exec_mode = ExecutionMode()
multi_proc_ctx = ExecutionContext(context=exec_mode.multi_proc)
run = Executor(exec_context=multi_proc_ctx, configs=configs)

In [8]:
i = 0
verbose = False
results = {}
for raw_result, tensor_field in run.execute():
    result = pd.DataFrame(raw_result)
    if verbose:
        print()
        print(f"Tensor Field: {type(tensor_field)}")
        print(tabulate(tensor_field, headers='keys', tablefmt='psql'))
        print(f"Output: {type(result)}")
        print(tabulate(result, headers='keys', tablefmt='psql'))
        print()
    results[i] = {}
    results[i]['result'] = result
    results[i]['simulation_parameters'] = simulation_parameters[i]
    i += 1

multi_proc: [<cadCAD.configuration.Configuration object at 0x108a7ee80>]


In [9]:
results[0]['result'][results[0]['result']['run']==1]

,center,run,size,substep,timestep
0,"[0, 1, 2, 3, 4]",1,5,0,0
1,"[0, 1, 2, 3, 4]",1,5,1,1
2,"[0, 2, 3]",1,6,2,1
3,"[0, 2, 3]",1,6,1,2
4,[3],1,7,2,2
5,[3],1,7,1,3
6,[3],1,8,2,3
7,[3],1,8,1,4
8,"[1, 3, 4]",1,9,2,4
9,"[1, 3, 4]",1,9,1,5


In [10]:
results[0]['result'][results[0]['result']['run']==2]

,center,run,size,substep,timestep
11,"[0, 1, 2, 3, 4]",2,5,0,0
12,"[0, 1, 2, 3, 4]",2,5,1,1
13,[4],2,11,2,1
14,[4],2,11,1,2
15,[4],2,12,2,2
16,[4],2,12,1,3
17,[4],2,13,2,3
18,[4],2,13,1,4
19,"[1, 3, 4, 13]",2,18,2,4
20,"[1, 3, 4, 13]",2,18,1,5


In [11]:
results[0]['result'][results[0]['result']['run']==3]

,center,run,size,substep,timestep
22,"[0, 1, 2, 3, 4]",3,5,0,0
23,"[0, 1, 2, 3, 4]",3,5,1,1
24,[4],3,14,2,1
25,[4],3,14,1,2
26,"[1, 3, 4]",3,15,2,2
27,"[1, 3, 4]",3,15,1,3
28,"[1, 3, 4]",3,16,2,3
29,"[1, 3, 4]",3,16,1,4
30,"[1, 3, 4]",3,17,2,4
31,"[1, 3, 4]",3,17,1,5


Notice how the size of the graph went from 13 to 18 in the second Monte Carlo run, indicating that the third run was executed in the middle of the second run